In [157]:
import pandas as pd

In [158]:
weather = pd.read_csv('delta_weather.csv')

In [159]:
Delta = pd.read_csv('delta_air_lines_flights.csv')

In [160]:
weather['Name'].unique()

array(['Atlanta, GA, United States', 'Charlotte, NC, United States',
       'Chicago, IL, United States', 'Denver, CO, United States',
       'Fort Worth Ave, Dallas, TX 75208, United States',
       'Houston, TX, United States', 'Los Angeles, CA, United States',
       'New York, NY, United States', 'Seattle, WA, United States',
       'Washington, DC, United States'], dtype=object)

In [161]:
my_lambda = lambda x: ''.join(x.split(',')[0]).lower()
weather['Name'] = weather['Name'].apply(my_lambda)

In [162]:
weather['Name'].replace('fort worth ave', 'dallas/fort worth', inplace=True)

In [163]:
weather['Name'].unique()

array(['atlanta', 'charlotte', 'chicago', 'denver', 'dallas/fort worth',
       'houston', 'los angeles', 'new york', 'seattle', 'washington'],
      dtype=object)

In [164]:
weather['Date time'] = pd.to_datetime(weather['Date time'])
weather['Date time'] = weather['Date time'].dt.strftime('%Y-%m-%d')

In [165]:
weather_wvc = weather[['Name', 'Date time', 'Wind Speed', 'Visibility', 'Conditions']]

### Delta

In [166]:
Delta.head()

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,dep_time,arr_time,...,arr_delay,crs_elapsed_time,actual_elapsed_time,air_time,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-12-31,DL,12953,"New York, NY",13930,"Chicago, IL",1110,1257,1114.0,1228.0,...,-29.0,167.0,134.0,110.0,733.0,NaN,NaN,NaN,NaN,NaN
1,2019-12-31,DL,12892,"Los Angeles, CA",12478,"New York, NY",2140,556,2138.0,553.0,...,-3.0,316.0,315.0,284.0,2475.0,NaN,NaN,NaN,NaN,NaN
2,2019-12-31,DL,10397,"Atlanta, GA",12953,"New York, NY",2130,2338,2128.0,2321.0,...,-17.0,128.0,113.0,95.0,762.0,NaN,NaN,NaN,NaN,NaN
3,2019-12-31,DL,13930,"Chicago, IL",10397,"Atlanta, GA",1500,1757,1458.0,1739.0,...,-18.0,117.0,101.0,83.0,606.0,NaN,NaN,NaN,NaN,NaN
4,2019-12-31,DL,11278,"Washington, DC",10397,"Atlanta, GA",1339,1538,1329.0,1519.0,...,-19.0,119.0,110.0,89.0,547.0,NaN,NaN,NaN,NaN,NaN


In [167]:
Delta['origin_city_name'] = Delta['origin_city_name'].apply(my_lambda)
Delta['dest_city_name'] = Delta['dest_city_name'].apply(my_lambda)

In [168]:
### Adding columns based on origin city name
Delta_data = pd.merge(Delta, weather_wvc, how = 'left', left_on=['origin_city_name', 'fl_date'], right_on=['Name', 'Date time']).rename(columns={"Wind Speed": "origin_wind_speed",
                                                                                                                                            "Visibility": "origin_visibility",
                                                                                                                                            "Conditions": "origin_conditions"})

In [169]:
### Adding columns based on dest city name
Delta_data = pd.merge(Delta_data, weather_wvc, how = 'left', left_on=['dest_city_name', 'fl_date'], right_on=['Name', 'Date time']).rename(columns={"Wind Speed": "dest_wind_speed",
                                                                                                                                            "Visibility": "dest_visibility",
                                                                                                                                            "Conditions": "dest_conditions"})

In [170]:
Delta_data.tail()

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,dep_time,arr_time,...,Name_x,Date time_x,origin_wind_speed,origin_visibility,origin_conditions,Name_y,Date time_y,dest_wind_speed,dest_visibility,dest_conditions
305931,2018-01-01,DL,12892,los angeles,12478,new york,615,1435,614.0,1410.0,...,los angeles,2018-01-01,5.6,4.4,Clear,new york,2018-01-01,12.9,9.9,Clear
305932,2018-01-01,DL,12892,los angeles,14747,seattle,2035,2337,2028.0,2255.0,...,los angeles,2018-01-01,5.6,4.4,Clear,seattle,2018-01-01,7.0,9.8,Partially cloudy
305933,2018-01-01,DL,10397,atlanta,13930,chicago,1702,1815,1700.0,1803.0,...,atlanta,2018-01-01,14.6,9.9,Partially cloudy,chicago,2018-01-01,12.7,9.9,Clear
305934,2018-01-01,DL,12892,los angeles,14747,seattle,730,1033,726.0,1035.0,...,los angeles,2018-01-01,5.6,4.4,Clear,seattle,2018-01-01,7.0,9.8,Partially cloudy
305935,2018-01-01,DL,13930,chicago,10397,atlanta,705,1010,704.0,948.0,...,chicago,2018-01-01,12.7,9.9,Clear,atlanta,2018-01-01,14.6,9.9,Partially cloudy


In [171]:
Delta_data = Delta_data.drop(['Name_x', 'Date time_x', 'Name_y', 'Date time_y'], axis = 1)

In [172]:
Delta_data.columns

Index(['fl_date', 'mkt_carrier', 'origin_airport_id', 'origin_city_name',
       'dest_airport_id', 'dest_city_name', 'crs_dep_time', 'crs_arr_time',
       'dep_time', 'arr_time', 'dep_delay', 'arr_delay', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'distance', 'carrier_delay',
       'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay',
       'origin_wind_speed', 'origin_visibility', 'origin_conditions',
       'dest_wind_speed', 'dest_visibility', 'dest_conditions'],
      dtype='object')

In [173]:
Delta_data['day_of_the_week'] = pd.to_datetime(Delta_data['fl_date']).dt.day_name()

In [174]:
Delta_data['fl_date'] = pd.to_datetime(Delta_data['fl_date'])

In [175]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()

In [176]:
holidays = cal.holidays(start='2018-01-01', end='2020-01-01')

In [177]:
holidays = holidays.append(pd.Index(['2018-12-31','2019-12-31']))   

In [178]:
print(holidays)

Index([2018-01-01 00:00:00, 2018-01-15 00:00:00, 2018-02-19 00:00:00,
       2018-05-28 00:00:00, 2018-07-04 00:00:00, 2018-09-03 00:00:00,
       2018-10-08 00:00:00, 2018-11-12 00:00:00, 2018-11-22 00:00:00,
       2018-12-25 00:00:00, 2019-01-01 00:00:00, 2019-01-21 00:00:00,
       2019-02-18 00:00:00, 2019-05-27 00:00:00, 2019-07-04 00:00:00,
       2019-09-02 00:00:00, 2019-10-14 00:00:00, 2019-11-11 00:00:00,
       2019-11-28 00:00:00, 2019-12-25 00:00:00, 2020-01-01 00:00:00,
              '2018-12-31',        '2019-12-31'],
      dtype='object')


In [179]:
Delta_data['holiday'] = Delta_data['fl_date'].isin(holidays)

In [180]:
Delta_data

,fl_date,mkt_carrier,origin_airport_id,origin_city_name,dest_airport_id,dest_city_name,crs_dep_time,crs_arr_time,dep_time,arr_time,...,security_delay,late_aircraft_delay,origin_wind_speed,origin_visibility,origin_conditions,dest_wind_speed,dest_visibility,dest_conditions,day_of_the_week,holiday
0,2019-12-31,DL,12953,new york,13930,chicago,1110,1257,1114.0,1228.0,...,NaN,NaN,14.4,9.6,"Rain, Overcast",25.3,7.9,"Snow, Overcast",Tuesday,True
1,2019-12-31,DL,12892,los angeles,12478,new york,2140,556,2138.0,553.0,...,NaN,NaN,4.2,9.9,Clear,14.4,9.6,"Rain, Overcast",Tuesday,True
2,2019-12-31,DL,10397,atlanta,12953,new york,2130,2338,2128.0,2321.0,...,NaN,NaN,18.6,9.9,Clear,14.4,9.6,"Rain, Overcast",Tuesday,True
3,2019-12-31,DL,13930,chicago,10397,atlanta,1500,1757,1458.0,1739.0,...,NaN,NaN,25.3,7.9,"Snow, Overcast",18.6,9.9,Clear,Tuesday,True
4,2019-12-31,DL,11278,washington,10397,atlanta,1339,1538,1329.0,1519.0,...,NaN,NaN,15.3,8.7,Overcast,18.6,9.9,Clear,Tuesday,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305931,2018-01-01,DL,12892,los angeles,12478,new york,615,1435,614.0,1410.0,...,NaN,NaN,5.6,4.4,Clear,12.9,9.9,Clear,Monday,True
305932,2018-01-01,DL,12892,los angeles,14747,seattle,2035,2337,2028.0,2255.0,...,NaN,NaN,5.6,4.4,Clear,7.0,9.8,Partially cloudy,Monday,True
305933,2018-01-01,DL,10397,atlanta,13930,chicago,1702,1815,1700.0,1803.0,...,NaN,NaN,14.6,9.9,Partially cloudy,12.7,9.9,Clear,Monday,True
305934,2018-01-01,DL,12892,los angeles,14747,seattle,730,1033,726.0,1035.0,...,NaN,NaN,5.6,4.4,Clear,7.0,9.8,Partially cloudy,Monday,True


In [181]:
Delta_data.to_csv('Delta_data.csv', sep=",", index = False)